# BioExplorer - Atlas
![](../bioexplorer_atlas_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3, Quaternion

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

load_cells = False
load_meshes = True

### Load Atlas

In [ ]:
if load_cells:
    atlas_assembly_name = 'AtlasCells'
    be.remove_assembly(atlas_assembly_name)
    atlas_assembly = be.add_assembly(atlas_assembly_name)
    atlas_model = be.add_atlas(
        assembly_name=atlas_assembly_name,
        population_name='atlas_ccfv3',
        load_cells=True, cell_radius=20.0,
        load_meshes=False,
        region_sql_filter='level>8'
    )

In [ ]:
if load_meshes:
    atlas_assembly_name = 'AtlasMeshes'
    be.remove_assembly(atlas_assembly_name)
    atlas_assembly = be.add_assembly(atlas_assembly_name)
    atlas_model = be.add_atlas(
        assembly_name=atlas_assembly_name,
        population_name='atlas_ccfv3',
        load_cells=False, load_meshes=True,
        region_sql_filter='level>=5 and guid%10=0',
        mesh_scale=Vector3(1, 1, 1)
    )

In [ ]:
be.reset_camera()
core.set_renderer()

In [ ]:
status = core.set_renderer(
    current='advanced',
    subsampling=4, max_accum_frames=64)
params = core.AdvancedRendererParams()
params.gi_ray_length = 1000.0
params.shadow_intensity = 1.0
params.soft_shadow_strength = 1.0
params.main_exposure = 2.0
status = core.set_renderer_params(params)

In [ ]:
import json
import os
atlas_data_folder = os.getenv('ATLAS_DATA_FOLDER')
data_folder = os.path.join(atlas_data_folder, 'mouse', 'v2')
f = open(os.path.join(data_folder, 'brain_regions.json'))
obj = json.load(f)

colors=dict()
colors[0] = 'FFFFFF'
def display_child(node):
    node_id = node['id']
    color = node['color_hex_triplet']
    colors[node_id] = color
    for child in node['children']:
        display_child(child)

for node in obj['msg']:
    display_child(node)

In [ ]:
def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

model_ids = be.get_model_ids()['ids']
for model_id in model_ids:
    material_ids = be.get_material_ids(model_id)['ids'][:-1]
    nb_materials = len(material_ids)
    palette = list()
    opacities = list()
    shading_modes = list()
    for material_id in material_ids:
        c = hex_to_rgb(colors[material_id])
        palette.append([c[0] / 255.0, c[1] / 255.0, c[2] / 255.0])
        if model_id == 26:
            shading_modes.append(be.shading_mode.NONE)
            opacities.append(1.0)
        else:
            shading_modes.append(be.shading_mode.DIFFUSE)
            opacities.append(1.25)
    be.set_materials(
        opacities=opacities,
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=palette, specular_colors=palette)
core.set_renderer()

In [ ]:
# Streamlines transformations
transformation = {
    'rotation': [1,0,0,0],
    'rotation_center': [0.0, 0.0, 0.0],
    'scale': [1.0, 1.0, 1.0],
    'translation': [-6500, 3500,5500]
}
core.update_model(43, transformation=transformation)